In [1]:
%cd /content/drive/My Drive/Colab Notebooks/playing with btc

/content/drive/My Drive/Colab Notebooks/playing with btc


In [ ]:
!pip install backtrader
!pip install pyfolio
# For plotting capabilities
!pip install backtrader[plotting]

In [4]:
%matplotlib inline
import pandas as pd
import backtrader as bt
import backtrader.indicators as btind
import backtrader.analyzers as btanalyzers
import datetime

from backtrader.feeds import PandasData



In [5]:
df = pd.read_csv('bitcoin/btc_4h_2017_2020.csv')
names = ['open', 'high', 'low', 'close', 'volume', 'open_time']
df = df[names]
df

,open,high,low,close,volume,open_time
0,4261.48,4349.99,4261.32,4349.99,82.088865,2017-08-17 04:00:00.000
1,4333.32,4485.39,4333.32,4427.30,63.619882,2017-08-17 08:00:00.000
2,4436.06,4485.39,4333.42,4352.34,174.562001,2017-08-17 12:00:00.000
3,4352.33,4354.84,4200.74,4325.23,225.109716,2017-08-17 16:00:00.000
4,4307.56,4369.69,4258.56,4285.08,249.769913,2017-08-17 20:00:00.000
...,...,...,...,...,...,...
6775,10449.94,10502.79,10360.00,10372.90,7823.399186,2020-09-22 04:00:00.000
6776,10372.91,10503.00,10372.90,10452.26,8386.425935,2020-09-22 08:00:00.000
6777,10452.11,10520.00,10416.00,10451.07,9044.594223,2020-09-22 12:00:00.000
6778,10451.06,10506.22,10447.53,10491.97,6701.006892,2020-09-22 16:00:00.000


# ticker and the start and end dates for testing

In [6]:
format = '%Y-%m-%d %H:%M:%S.%f'
df['open_time'] = pd.to_datetime(df['open_time'], format=format)
df.set_index('open_time', inplace=True)
df.head(2)

,open,high,low,close,volume
open_time,,,,,
2017-08-17 04:00:00,4261.48,4349.99,4261.32,4349.99,82.088865
2017-08-17 08:00:00,4333.32,4485.39,4333.32,4427.30,63.619882


In [7]:
# class to define the columns we will provide
class SignalData(PandasData):
    """
    Define pandas DataFrame structure
    """
    OHLCV = ['open', 'high', 'low', 'close', 'volume']
    cols = OHLCV

    # create lines
    lines = tuple(cols)

    # define parameters
    params = {c: -1 for c in cols}
    params.update({'datetime': None})
    params = tuple(params.items())

# Strategy

Use two moving averages and buy in the cross of the moving average, but the RSI must be above a certain range.

Then sell the position when the RSI is too high or when we achieve a certain value.

In [8]:
# define backtesting strategy class
class Strategy(bt.Strategy):
  params = dict()
  
  def __init__(self):

    self.btc = self.datas[0]
    self.close = self.btc.close
    self.ema_20 = btind.ExponentialMovingAverage(self.btc.close, period=20)
    self.ema_40 = btind.ExponentialMovingAverage(self.btc.close, period=40)
    # 1 if 20 crosses up 40 -1 if 20 crosses down 40
    self.cross = btind.CrossOver(self.ema_20, self.ema_40)

    self.order = None

  # logging function
  def log(self, txt):
    '''Logging function'''
    dt = self.datas[0].datetime.date(0).isoformat()
    print(f'{dt}, {txt}')

  def next(self):
    # not in the market
    if not self.position:  # not in the market
      if self.cross[0] > 0:
        self.order = self.buy()
        self.log(f'Buy at {self.btc.close[0]}')
        self.order = 'buying'

    elif self.order == 'buying': # position in market
      if self.cross[0] < 0:
        self.order = self.sell()
        self.log(f'Sell at {self.btc.close[0]}')




In [9]:

# instantiate SignalData class
data = SignalData(dataname=df)

In [10]:
# instantiate Cerebro, add strategy, data, initial cash, commission and pyfolio for performance analysis
cerebro = bt.Cerebro(stdstats = False)
cerebro.addstrategy(Strategy)
cerebro.adddata(data, name='BTC/USDT')
cerebro.broker.setcash(20000.0)
cerebro.broker.setcommission(commission=0.001)
cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')

# Analizers

In [11]:
# Analyzer
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='ta')
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe', riskfreerate=0.0, annualize=True, timeframe=bt.TimeFrame.Minutes)
cerebro.addanalyzer(bt.analyzers.VWR, _name='vwr')
cerebro.addanalyzer(bt.analyzers.SQN, _name='sqn')
cerebro.addanalyzer(bt.analyzers.Transactions, _name='txn')

# Observers


In [12]:
cerebro.addobserver(bt.observers.TimeReturn)
cerebro.addobserver(bt.observers.BuySell)
cerebro.addobserver(bt.observers.Value)
cerebro.addobserver(bt.observers.DrawDown)
cerebro.addobserver(bt.observers.Trades)

In [13]:
# run the backtest
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
backtest_result = cerebro.run()
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 20000.00
2017-08-25, Buy at 4312.69
2017-09-04, Sell at 4153.0
2017-09-07, Buy at 4613.94
2017-09-09, Sell at 4242.01
2017-09-19, Buy at 3965.01
2017-09-21, Sell at 3605.03
2017-09-25, Buy at 3920.75
2017-10-25, Sell at 5541.51
2017-10-26, Buy at 5834.38
2017-11-10, Sell at 6506.98
2017-11-15, Buy at 7182.0
2017-12-21, Sell at 16490.99
2018-01-03, Buy at 14730.05
2018-01-09, Sell at 14400.0
2018-01-28, Buy at 12012.62
2018-01-30, Sell at 11118.99
2018-02-12, Buy at 8903.0
2018-02-23, Sell at 10033.0
2018-02-27, Buy at 10568.01
2018-03-07, Sell at 9844.18
2018-03-21, Buy at 8931.01
2018-03-25, Sell at 8470.15
2018-04-12, Buy at 7690.0
2018-05-08, Sell at 9255.01
2018-06-02, Buy at 7623.1
2018-06-05, Sell at 7428.88
2018-06-05, Buy at 7621.01
2018-06-10, Sell at 7261.03
2018-06-20, Buy at 6761.51
2018-06-22, Sell at 6348.52
2018-06-30, Buy at 6390.07
2018-07-10, Sell at 6296.91
2018-07-16, Buy at 6676.48
2018-08-01, Sell at 7539.99
2018-08-17, Buy at 6584.49
201

In [14]:
strat = backtest_result[0]
from pprint import pprint

drawdown = strat.analyzers.drawdown.get_analysis()
pprint(drawdown)
sharpe_ratio = strat.analyzers.sharpe.get_analysis()
pprint(sharpe_ratio)
transactions = strat.analyzers.txn.get_analysis()
pprint(transactions)





AutoOrderedDict([('len', 2723),
                 ('drawdown', 9.921463971749978),
                 ('moneydown', 3780.291679999973),
                 ('max',
                  AutoOrderedDict([('len', 3289),
                                   ('drawdown', 16.697548446108936),
                                   ('moneydown', 5869.90638)]))])
OrderedDict([('sharperatio', 0.022621616375405346)])
OrderedDict([(datetime.datetime(2017, 8, 25, 4, 0),
              [[1, 4312.69, 0, 'BTC/USDT', -4312.69]]),
             (datetime.datetime(2017, 9, 4, 12, 0),
              [[-1, 4153.0, 0, 'BTC/USDT', 4153.0]]),
             (datetime.datetime(2017, 9, 7, 12, 0),
              [[1, 4616.79, 0, 'BTC/USDT', -4616.79]]),
             (datetime.datetime(2017, 9, 9, 4, 0),
              [[-1, 4254.44, 0, 'BTC/USDT', 4254.44]]),
             (datetime.datetime(2017, 9, 19, 16, 0),
              [[1, 3970.0, 0, 'BTC/USDT', -3970.0]]),
             (datetime.datetime(2017, 9, 21, 20, 0),
              [

In [15]:
cerebro.plot()

<IPython.core.display.Javascript object>

[[<Figure size 432x288 with 7 Axes>]]